In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [2]:
prefix = ''
sessname =''
nclass = 1
epochs =2
mini_batch_size =25
lr = 0.001
lr_scheduler_factor =0.1
momentum =0.9
weight_decay =0.0005
overlap = 0.5
momentum = 0.45
weight_decay =0.0005
nms_thresh = 0.45
image_shape =256
label_width =150
n_train_samples = 16551
network ='resnet-50'
optim = 'sgd'
role = ''

In [3]:
sess = sagemaker.Session()
bucket = sess.default_bucket()

training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")
print (training_image)

  



ValueError: Algorithm class: doesn't have mapping to account_id with images